In [122]:
import json
import ast
import pandas as pd
from urllib.parse import urlparse
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from urllib.parse import urlparse

[nltk_data] Downloading package punkt to /Users/kippy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import nltk
nltk.download('punkt')

In [101]:
## load the data into a dataframe, spit out the row of an error if any
list_df = []
i = 0
for row in open("article_data.csv", 'r').readlines():
    url, data = row.split(', ', 1)
    try:
        data = ast.literal_eval(data)
        data["url"] = url
        list_df.append(data)
    except Exception as e:
        print(i, e)    
    i += 1 
df = pd.io.json.json_normalize(list_df)

In [102]:
# How many are invalid?
df[df.apply(lambda x: x.title is None or x.text is None, axis=1)].shape[0], df.shape[0]

(2570, 113540)

In [119]:
clean_df = df[df.apply(lambda x: x.title is not None and x.text is not None, axis=1)]

In [156]:
tqdm.pandas()
clean_df["title_tokenized"] = clean_df.progress_apply(lambda x: [token.lower() for token in nltk.word_tokenize(x.title)]
                                                      , axis = 1)
clean_df["text_tokenized"] = clean_df.progress_apply(lambda x: [token.lower() for token in nltk.word_tokenize(x.text)]
                                                     , axis = 1)
clean_df["domain"] = clean_df.progress_apply(lambda x: urlparse(x.url).netloc, axis = 1)
clean_df['candidate_title'] = clean_df.progress_apply(lambda x: candidates.intersection(x.title_tokenized), axis=1)
clean_df['candidate_text'] = clean_df.progress_apply(lambda x: candidates.intersection(x.text_tokenized), axis=1)

/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [158]:
clean_df.memory_usage().agg('sum')

22110320

In [153]:
clean_df.groupby('domain').agg('count').sort_values(by='authors', ascending=False).head(100)

,authors,date_download,date_modify,date_publish,description,filename,image_url,language,localpath,source_domain,text,title,title_page,title_rss,url,title_tokenized,text_tokenized
domain,,,,,,,,,,,,,,,,,
thehill.com,8072,8072,8072,8072,8015,8072,8072,291,0,8072,8072,8072,0,0,8072,8072,8072
www.cnn.com,5158,5158,5158,5158,5142,5158,5158,193,0,5158,5158,5158,0,0,5158,5158,5158
www.nytimes.com,4733,4733,4733,4733,4726,4733,4733,4733,0,4733,4733,4733,0,0,4733,4733,4733
www.politico.com,3160,3160,3160,3160,2965,3160,3160,3157,0,3160,3160,3160,0,0,3160,3160,3160
www.newsweek.com,2846,2846,2846,2846,2846,2846,2846,2846,0,2846,2846,2846,0,0,2846,2846,2846
www.reuters.com,2451,2451,2451,2451,2449,2451,2451,2441,0,2451,2451,2451,0,0,2451,2451,2451
www.nbcnews.com,2433,2433,2433,2433,2266,2433,2433,2266,0,2433,2433,2433,0,0,2433,2433,2433
www.theguardian.com,2280,2280,2280,2280,2280,2280,2280,2280,0,2280,2280,2280,0,0,2280,2280,2280
www.foxnews.com,2020,2020,2020,2020,1996,2020,2020,2020,0,2020,2020,2020,0,0,2020,2020,2020


In [145]:
candidates = [ "TRUMP", "SANDERS","DELANEY","WARREN","HARRIS","GILLIBRAND","O'ROURKE","KLOBUCHAR","BOOKER",
    "BUTTIGIEG","GABBARD","YANG","INSLEE","HICKENLOOPER","WILLIAMSON","DE LA FUENTE","CASTRO","BIDEN"]
candidates = set([x.lower() for x in candidates])
candidates

{'biden',
 'booker',
 'buttigieg',
 'castro',
 'de la fuente',
 'delaney',
 'gabbard',
 'gillibrand',
 'harris',
 'hickenlooper',
 'inslee',
 'klobuchar',
 "o'rourke",
 'sanders',
 'trump',
 'warren',
 'williamson',
 'yang'}

In [172]:
candidate_counts_title = {}
for item in clean_df.apply(lambda x: tuple(x.candidate_title), axis=1):
    candidate_counts_title[item] = candidate_counts_title.get(item, 0) + 1
candidate_counts_title

{('trump',): 36578,
 (): 62960,
 ('trump', 'warren'): 201,
 ('warren',): 1144,
 ('sanders',): 2361,
 ("o'rourke",): 442,
 ('harris',): 818,
 ('gabbard',): 301,
 ('biden',): 1925,
 ('trump', 'castro'): 23,
 ('trump', 'sanders'): 453,
 ('inslee',): 198,
 ('klobuchar',): 206,
 ('sanders', 'warren'): 75,
 ('booker',): 244,
 ('castro',): 92,
 ('harris', 'biden'): 21,
 ('sanders', 'biden'): 195,
 ('trump', 'biden'): 404,
 ('trump', "o'rourke"): 96,
 ('booker', 'harris'): 19,
 ('trump', 'harris'): 120,
 ('trump', 'sanders', 'biden'): 39,
 ('yang',): 290,
 ('trump', 'klobuchar'): 6,
 ('klobuchar', 'sanders', 'trump'): 1,
 ('gillibrand',): 174,
 ('gabbard', 'sanders', 'warren'): 3,
 ('gabbard', 'trump'): 20,
 ('delaney',): 34,
 ('sanders', 'harris', 'biden'): 5,
 ('trump', 'booker'): 25,
 ('klobuchar', 'trump', 'booker', 'harris'): 1,
 ('gillibrand', 'harris'): 2,
 ('gabbard', 'gillibrand'): 2,
 ('trump', 'gillibrand'): 22,
 ('klobuchar', 'trump'): 22,
 ('harris', 'warren'): 13,
 ("o'rourke", '

In [173]:
candidate_counts_text = {}
for item in clean_df.apply(lambda x: tuple(x.candidate_text), axis=1):
    candidate_counts_text[item] = candidate_counts_text.get(item, 0) + 1
candidate_counts_text

{('trump',): 55573,
 (): 29612,
 ('trump', 'warren'): 1345,
 ('harris',): 680,
 ('trump', 'harris'): 803,
 ('trump', 'sanders', 'biden', 'warren'): 77,
 ('trump', 'buttigieg'): 386,
 ("o'rourke", 'sanders'): 28,
 ('trump', 'sanders'): 4057,
 ('sanders', 'warren'): 182,
 ('sanders', 'biden', 'warren'): 29,
 ("o'rourke",
  'booker',
  'castro',
  'trump',
  'biden',
  'sanders',
  'warren',
  'gabbard'): 2,
 ('booker', 'trump', 'biden', 'harris', 'warren'): 10,
 ('gillibrand',
  'booker',
  'castro',
  'delaney',
  'trump',
  'hickenlooper',
  'biden',
  'sanders',
  'harris',
  'warren',
  'klobuchar',
  'buttigieg'): 3,
 ('buttigieg', "o'rourke"): 16,
 ('gillibrand', 'booker', 'trump', 'biden', 'sanders', 'harris', 'warren'): 56,
 ("o'rourke", 'sanders', 'biden'): 11,
 ('booker', 'sanders', 'biden', 'warren'): 1,
 ('trump', 'sanders', 'warren'): 425,
 ('sanders',): 1242,
 ('trump', 'buttigieg', 'biden'): 85,
 ("o'rourke", 'castro', 'delaney', 'biden', 'sanders', 'warren', 'yang'): 1,
 